<a href="https://colab.research.google.com/github/06yako14/stock-prediction/blob/dev/src/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ニューラルネットワークによる日経平均株価予測

### 使用ライブラリのインポート

In [2]:
import datetime

import numpy as np
import pandas as pd

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import matplotlib.pyplot as plt
try:
  import japanize_matplotlib
except:
  !pip install japanize-matplotlib
  import japanize_matplotlib

import yfinance as yf

pd.options.display.max_columns=50
pd.options.display.max_rows=100

%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=45cb93ea2c2cb252501b4b5f19c558bcde32e5b6dd1ba33c1ca7925fe6bdd68c
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize-matplotlib


### データの読み込み

In [13]:
# 取得日付を指定
start = datetime.datetime(1965, 1, 5)
end = datetime.date.today()

# 銘柄コード、ティッカーシンボルの設定
# 今回は日経平均株価(日経225)
stock_code = '^N225'

# 株価取得
stock_price = yf.download(tickers=stock_code, interval='1d', period='max')

[*********************100%%**********************]  1 of 1 completed


### データの確認

In [14]:
stock_price.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1965-01-05,1257.719971,1257.719971,1257.719971,1257.719971,1257.719971,0
1965-01-06,1263.989990,1263.989990,1263.989990,1263.989990,1263.989990,0
1965-01-07,1274.270020,1274.270020,1274.270020,1274.270020,1274.270020,0
1965-01-08,1286.430054,1286.430054,1286.430054,1286.430054,1286.430054,0
1965-01-12,1288.540039,1288.540039,1288.540039,1288.540039,1288.540039,0


### 欠損値の確認

In [15]:
stock_price.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

### データの前処理

#### 不要行の削除

In [37]:
# 出来高集計以前のデータを削除
stock_price_val = stock_price[stock_price['Volume'] != 0]

#### 特徴量エンジニアリング

In [33]:
# 単純移動平均 Simple Moving Average (SMA)
# 終値の単純移動平均を計算
def simpleMovingAverage(df, col, spanlist):
    for span in spanlist:
        column_name = f"{span}SMA"
        new_col = df[col].rolling(span, min_periods=1).mean()
        df = pd.concat([df, new_col.rename(column_name)], axis=1)
    return df

In [42]:
sma_spanlist = [5, 20, 50, 200]
sp_sma = pd.DataFrame()
sp_sma = simpleMovingAverage(stock_price_val, 'Adj Close', sma_spanlist)

### データセットの作成

#### 目的変数の作成

In [44]:
sp_sma['Next_Close'] = sp_sma['Adj Close'].shift(-1)

In [45]:
sp_sma.head()

,Open,High,Low,Close,Adj Close,Volume,5SMA,20SMA,50SMA,200SMA,Next_Close
Date,,,,,,,,,,,
2002-06-10,11470.919922,11522.040039,11370.209961,11370.209961,11370.209961,37900000,11370.209961,11370.209961,11370.209961,11370.209961,11449.440430
2002-06-11,11390.410156,11514.530273,11390.410156,11449.440430,11449.440430,39600000,11409.825195,11409.825195,11409.825195,11409.825195,11327.059570
2002-06-12,11392.320312,11405.290039,11261.929688,11327.059570,11327.059570,42800000,11382.236654,11382.236654,11382.236654,11382.236654,11144.839844
2002-06-13,11366.059570,11396.280273,11132.589844,11144.839844,11144.839844,46500000,11322.887451,11322.887451,11322.887451,11322.887451,10920.629883
2002-06-14,11121.889648,11127.160156,10911.070312,10920.629883,10920.629883,112800000,11242.435938,11242.435938,11242.435938,11242.435938,10664.110352
